In [1]:
import cv2

In [2]:
# Read image
img = cv2.imread("/home/said/courses/Projetinfo/sports_scr/data/natation/000000.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect text regions using Otsu thresholding
thresh = cv2.threshold(gray, 0, 100, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Find contours
contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

# Initialize the list of bounding boxes
bounding_boxes = []

# Iterate through the contours
for c in contours:
    # Get the bounding box of the contour
    x, y, w, h = cv2.boundingRect(c)
    bounding_boxes.append((x, y, w, h))

In [3]:
# Draw all bounding boxes on the image
for b in bounding_boxes:
    x, y, w, h = b
    img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Show the image
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/said/.local/lib/python3.10/site-packages/cv2/qt/plugins"


In [4]:
import numpy as np

# Get the original image size
height, width = img.shape[:2]

# Create a blob from the image
blob = cv2.dnn.blobFromImage(img, 1.0, (320, 320), (123.68, 116.78, 103.94), swapRB=True, crop=False)

# Load the EAST text detector
net = cv2.dnn.readNet("/home/said/courses/Projetinfo/sports_scr/frozen_east_text_detection.pb")

# Set the input to the EAST text detector
net.setInput(blob)

# Get the output from the EAST text detector
scores, geometry = net.forward(["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"])

# Get the bounding boxes
(numRows, numCols) = scores.shape[2:4]
rects = []
confidences = []

# Apply non-maximum suppression to the bounding boxes
for y in range(0, numRows):
    scoresData = scores[0, 0, y]
    xData0 = geometry[0, 0, y]
    xData1 = geometry[0, 1, y]
    xData2 = geometry[0, 2, y]
    xData3 = geometry[0, 3, y]
    anglesData = geometry[0, 4, y]

    for x in range(0, numCols):
        if scoresData[x] < 0.1:
            continue

        (offsetX, offsetY) = (x * 4.0, y * 4.0)

        angle = anglesData[x]
        cos = np.cos(angle)
        sin = np.sin(angle)

        h = xData0[x] + xData2[x]
        w = xData1[x] + xData3[x]

        endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
        endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
        startX = int(endX - w)
        startY = int(endY - h)

        # Scale the bounding box coordinates
        startX = int(startX * width / 320)
        startY = int(startY * height / 320)
        endX = int(endX * width / 320)
        endY = int(endY * height / 320)

        rects.append((startX, startY, endX, endY))
        confidences.append(scoresData[x])

#boxes = cv2.dnn.NMSBoxes(rects, confidences, 0.5, 0.3)

# Draw the bounding boxes on the image
for box in rects:
    (startX, startY, endX, endY) = box
    cv2.rectangle(img, (startX, startY), (endX, endY), (0, 255, 0), 2)

# Show the image
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# Find the contour that contains the majority of the bounding boxes
majority_contour = None
max_overlap = 0

for c in contours:
    overlap = 0
    for b in bounding_boxes:
        x, y, w, h = b
        if cv2.pointPolygonTest(c, (x + w/2, y + h/2), False) >= 0:
            overlap += 1
    if overlap > max_overlap:
        max_overlap = overlap
        majority_contour = c

# Draw the majority contour on the image
img = cv2.drawContours(img, [majority_contour], -1, (255, 0, 0), 3)

# Show the image
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
import cv2

# Read image
img = cv2.imread("/home/said/courses/Projetinfo/sports_scr/data/natation/000000.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Detect text regions using Otsu thresholding
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Find contours
contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

# Initialize the list of bounding boxes
bounding_boxes = []

# Iterate through the contours
for c in contours:
    # Get the bounding box of the contour
    x, y, w, h = cv2.boundingRect(c)
    bounding_boxes.append((x, y, w, h))

# Calculate the total area of all bounding boxes
total_area = sum(w*h for x, y, w, h in bounding_boxes)

# Sort the bounding boxes by their area in decreasing order
bounding_boxes.sort(key=lambda x: x[2]*x[3], reverse=True)

# Initialize the list of selected bounding boxes
selected_boxes = []

# Initialize the total area of the selected bounding boxes
selected_area = 0

# Iterate through the sorted bounding boxes
for x, y, w, h in bounding_boxes[4:]:
    # Add the bounding box to the list of selected bounding boxes
    selected_boxes.append((x, y, w, h))
    selected_area += w*h

    # If the total area of the selected bounding boxes exceeds 95% of the total area of all bounding boxes, break the loop
    if selected_area / total_area >= 0.95:
        break

# Find the bounding box that surrounds all the selected bounding boxes
x_min = min(x for x, y, w, h in selected_boxes)
y_min = min(y for x, y, w, h in selected_boxes)
x_max = max(x + w for x, y, w, h in selected_boxes)
y_max = max(y + h for x, y, w, h in selected_boxes)

# Draw the bounding box on the image
img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

# Show the image
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()